# Práctica Métodos de Simulación - Parte 1

## Elena Rivas, Teresa Grau, Ignacio Casso

**Enunciado. Generación de números y variables aleatorias**. Utilizando el generador congruencial multiplicativo de IMSL

$~~~~~~~~~~x_{i+1}=16807x_i~mod~(2^{31}-1)$

obtener una muestra de números aleatorios en (0, 1) y estudiar utilizando los contrastes de la librería Diehard para la aleatoriedad (explicando brevemente en qué consisten 3 de ellos). 

### Generadores congruenciales

Un generador lineal congruencial (GLC) es un algoritmo que permite obtener una secuencia de números pseudoaleatorios calculados con una función lineal definida a trozos discontinua. Es uno de los métodos más antiguos y conocidos para la generación de números pseudoaleatorios. Siguen la siguiente fórmula de recursión:

$~~~~~~~~~~x_{n+1}=(ax_n+b)~mod~m$

que proporciona números enteros en [0, m), para un multiplicador *a*, sesgo *b*, módulo *m* y semilla x0 , donde suponemos que *a, b* ∈ {0, 1, ..., *m*-1}. La utilización de la misma semilla llevará a la misma secuencia de nos aleatorios (reproducibilidad y mutabilidad).

Se convierten en números uniformes en [0,1) dividiendo por *m*,

$~~~~~~~~~~u_n=\frac{x_n}{m}$

Si b = 0, se denominan multiplicativos, que es lo que nos pide el enunciado de la práctica:

$~~~~~~~~~~x_{n+1}=ax_n~mod~m$

### Generador MINSTD

Uno de los generadores congruenciales multiplicativos o de Lehmer más utilizados es el de parámetros $a = 16807$, $b = 2^{31}-1$, justo los que indica el enunciado de la práctica. A este generador se lo conoce comúnmente como MINSTD, y es todavía un estándar a persar de sufrir los defectos propios de los generadores congruenciales. Entre otros se usa en la libraría IMSL como se indica en la práctica, y en la librería estándar de C++. El parámetro *b* es elegido por ser a la vez el mayor número primo representable con 32 bits y por dar lugar por tanto al periodo máximo que puede alcanzar un generador de Lehmer con 32 bits, y además por ser posible implementar la operación módulo eficientemente con ese número. El parámetro *a* se escoge entre las raíces primitivas de *b* de nuevo para dar lugar a un periodo máximo, y entre ellas se escogió 16807 por sus buenos resultados, tanto en aleatoriedad como en eficiencia de implementación, a pesar de que posteriormente se demostró que existían mejores constantes, como por ejemplo 48271.

A continuación se genera una muestra de números aleatorios con este generador, empezando con una semilla de valor 1. Primero usamos el generador tal cual obteniendo la muestra de números naturales, y posteriormente mostramos la sucesión de numeros reales distribuidos uniformemente en (0,1) que se obtiene a partir de la primera muestra.

Hemos implementado MINSTD directamente en bash para incorporarlo más fácilmente al notebook, y no usamos el método de Schrage ya que al no haber desbordamiento no es estrictamente necesario.

##### Muestra de números naturales distribuidos aleatoriamente en (1,2^31-1)

In [4]:
SEED=1
a=16807
let m=0x7fffffff # 2^31-1
for i in {1..10}; do
    let SEED=a*SEED%m # This operation can be optimized in real implementations
    echo $SEED
done

16807
282475249
1622650073
984943658
1144108930
470211272
101027544
1457850878
1458777923
2007237709


Una forma de probar que la implementeción es correcta es empezar con una semilla de valor 1 y comprobar que tras generar 10000 números se obtiene 1043618065

In [3]:
SEED=1
a=16807
let m=0x7fffffff # 2^31-1
for i in {1..10000}; do
    let SEED=a*SEED%m # This operation can be optimized in real implementations
done
echo $SEED

1043618065


##### Muestra de números reales distribuidos aleatoriamente en (0,1)

Para obterner una muestra de numeros reales uniforme en (0,1), basta con dividir los números generados entre el módulo del generador, *m*.

In [6]:
SEED=1
a=16807
let m=0x7fffffff # 2^31-1
for i in {1..10}; do
    let SEED=a*SEED%m
    REAL=`echo "$SEED/$m" | bc -l`
    echo $REAL
done

.00000782636925942561
.13153778814316624223
.75560532219503322718
.45865013192344928715
.53276723741216922058
.21895918632809034843
.04704461621448612595
.67886471686831895116
.67929640583661217514
.93469289594082762298


##### Implementación de MINSTD usando el método de Scharage

Para completar esta sección, mostramos la implemetación de MINSTD usando el método de Schrage, necesaria en lenguajes de programación con enteros de 32-bits como máximo. Consúltese (Park y Miller, 1988) para una aclaración de la teoría detrás de este método.

In [10]:
SEED=1
a=16807
let m=0x7fffffff # 2^31-1
let q=m/a
let r=m%a
for i in {1..10000}; do
    let q2=SEED/q
    let r2=SEED%q
    let SEED=a*r2-r*q2
    if [ $SEED -lt 0 ]; then
        let SEED=SEED+m
    fi
done
echo $SEED

1043618065


## Librería Dieharder

La librería Diehard que menciona el enunciado es una batería de contrastes de aleatoriedad desarrollada por George Margsalia para establecer la calidad de un generador aleatorio. Nosotros usaremos en su lugar la librería Dieharder, más completa y que en particular incluye todos los tests de la librería Diehard original.

### Contrastes de aleatoriedad en Dieharder

Como se ha dicho, el objetivo de los tests es establecer la calidad de un generador aleatorio. Para ello todos los tests siguen el siguiente procedimiento común: toman una muestra de números generados por el rng en cuestión y realizan un contraste de hipótesis sobre esa muestra, donde la hipótesis nula es que el rng es un generador de números aleatorios perfecto, es decir, la muestra proviene de una distribución uniforme. Obtienen como resultado un p-valor o estadístico, cuya distribución de probabilidad bajo la hipótesis nula es conocida, y el test falla si el p-valor obtenido es muy poco probable bajo esa distribución.

Los distintos tests se diferencian por lo tanto en el contraste de hipótesis aplicado, es decir, en la función que obtiene ese p-valor a partir de la muestra y en la distribución que ese p-valor debería seguir. En el caso de Dieharder, esa distribución es la uniforme, y se obtienen varios p-valores (100 por defecto) por cada test para comprobar que realmente se adaptan a esa distribución uniforme. A partir de ese conjunto de p-valores se obtiene un último estadístico mediante el test de Kolmogorov-Smirnov, de forma que si ese estadístico esta muy cerca de 0 o 1 se rechaza la hipótesis nula y se considera que el generador ha fallado el test.

Más adelante veremos ejemplos de todo esto, y explicaremos brevemente 3 de los tests usados en Diehard y Dieharder.

### Setup

Para instalar dieharder en linux, basta con escribir en consola el siguiente comando:
<br>
*```sudo apt-get install dieharder```*

Nota: dieharder debe instalarse para ejecutar la mayoría de lo que queda del notebook

### Usage

Dieharder funciona mediante comandos por consola. A continuación mostramos el funcionamiento y resultado de los comandos que hemos utilizado para realizar la práctica:

##### Run a test for a generator

*```dieharder -d testId -g rng```*, donde *testId* es el identificador del test y *rng* el identificador del generador.

Ejemplo:

In [59]:
dieharder -d 0 -g 0

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|   Seed   |
       borosh13|  2.89e+08  |1216113653|
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment
#=============================================================================#
   diehard_birthdays|   0|       100|     100|0.69718427|  PASSED  


En el ejemplo anterior hemos usado el "birthday spacings" test, y uno de los generadores estándar que viene incluído en Diehard. Para usar un generador definido por el usuario, hay que usar un fichero con la muestra de números aleatorios, o un pipe para pasarle la muestra por stdin. Como MINSTD está incluido por defecto, cuando probemos el generador usaremos el MINSTD de Dieharder directamente por comodidad. Las otras dos opciones dan demasiados problemas: en la primera debemos asegurar que la muestra es lo suficientemente grande para aplicar los tests, y en la segunda habría que manejar la señal SIGPIPE para dejar de generar cuando Dieharder deje de leer (en el shell por defecto un proceso para al recibirla, pero en el notebook parece que no).

##### Show all available tests

Podemos ver todos los test disponibles en la libreria Dieharder con el siguiente comando:

In [63]:
dieharder -l

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
Installed dieharder tests:
 Test Number	                     Test Name	              Test Reliability
  -d 0  	                  Diehard Birthdays Test	      Good
  -d 1  	                     Diehard OPERM5 Test	      Good
  -d 2  	          Diehard 32x32 Binary Rank Test	      Good
  -d 3  	            Diehard 6x8 Binary Rank Test	      Good
  -d 4  	                  Diehard Bitstream Test	      Good
  -d 5  	                            Diehard OPSO	   Suspect
  -d 6  	                       Diehard OQSO Test	   Suspect
  -d 7  	                        Diehard DNA Test	   Suspect
  -d 8  	      Diehard Count the 1s (stream) Test	      Good
  -d 9  	        Diehard Count the 1s Test (byte)	      Good
  -d 10  	                Diehard Parking Lot Te

Como vemos, los 17 primeros corresponden a los tests de la librería Diehard original

##### Show all available generators

De igual forma, podemos ver todos los generadores disponibles en la libreria:

In [5]:
dieharder -g -l

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
#    Id Test Name           | Id Test Name           | Id Test Name           #
#=============================================================================#
|   000 borosh13            |001 cmrg                |002 coveyou             |
|   003 fishman18           |004 fishman20           |005 fishman2x           |
|   006 gfsr4               |007 knuthran            |008 knuthran2           |
|   009 knuthran2002        |010 lecuyer21           |011 minstd              |
|   012 mrg                 |013 mt19937             |014 mt19937_1999        |
|   015 mt19937_1998        |016 r250                |017 ran0                |
|   018 ran1                |019 ran2                |020 ran3                |
|   021 rand                |022 rand48 

Como vemos, el generador que usamos en la práctica, MINSTD, es el 011. Veamos la muestra que genera con una semilla de valor 1 para asegurarnos, comparándola con la generada anteriormente por nostros. Indicamos el generador que queremos usar con el argumento *-g*. El argumento *-S* sirve para establecer la semilla, en este caso *seed* = 1. Y el argumento *-t* para elegir la cantidad de números a obtener, en este caso, 10.

In [65]:
dieharder -o -g 011 -S 1 -t 10

#==================================================================
# generator minstd  seed = 1
#==================================================================
type: d
count: 10
numbit: 32
     16807
 282475249
1622650073
 984943658
1144108930
 470211272
 101027544
1457850878
1458777923
2007237709


##### Tests output

Se puede controlar el nivel de detalle de la salida mediante algunos flags usando la opción -D. Se pueden consultar las flags usando la siguiente opción.

In [2]:
dieharder -F 

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
 Dieharder output control flags.  Set with -D flagname or -D flag
 Multiple flags may be given:		dieharder -a -D test_name -D pvalues.
 Flags names and numbers may be mixed:	dieharder -a -D 8 -D pvalues.
 Flag	    Flag Name
    0	        default
    1	         header
    2	       show_rng
    4	    line_header
    8	      test_name
   16	         ntuple
   32	       tsamples
   64	       psamples
  128	        pvalues
  256	     assessment
  512	         prefix
 1024	    description
 2048	      histogram
 4096	           seed
 8192	           rate
16384	       show_num
32768	  no_whitespace


Algunos de las flags seleccionadas por defecto son las siguientes:

-  *tsamples*: tamaño de la muestra de números aleatorios utilizada para cada test y cada pvalue. Dependiendo del test puede ser fija o el test será más preciso cuanto más grande sea esa muestra.

-  *psamples*: número de p-valores que computa el test, es decir, número de veces que se repite el test. Por defecto es 100, y cuanto más grande más probable sera que un generador no totalmente aleatorio falle el test. Tambiém será más probable que un buen generador falle un test no del todo preciso, que es justo lo que pasa con algunos de los test de Diehard.

- *pvalues*: el p-valor o estadístico final que se obtiene a partir de los p-valores del test y que se usa para decidir si el test falla o no.

- *assessment*: resultado del test. Por defecto el generador falla el test si el p-valor es menor que 0.0005 o mayor que 0.9995, se condisedera débil si el p-valor es menor que 0.005 o mayor que 0.995, y supera el test en otro caso.

- *ntuples*: algunos test agrupan los números de la muestra en tuplas, y este valor muestra el tamaño de esas tuplas.

Otras flags particularmente interesantes son:

In [1]:
dieharder -g 0 -d 0 -D histogram -D description


#==================================================================
#                Diehard "Birthdays" test (modified).
# Each test determines the number of matching intervals from 512
# "birthdays" (by default) drawn on a 24-bit "year" (by
# default).  This is repeated 100 times (by default) and the
# results cumulated in a histogram.  Repeated intervals should be
# distributed in a Poisson distribution if the underlying generator
# is random enough, and a a chisq and p-value for the test are
# evaluated relative to this null hypothesis.
#
# It is recommended that you run this at or near the original
# 100 test samples per p-value with -t 100.
#
# Two additional parameters have been added. In diehard, nms=512
# but this CAN be varied and all Marsaglia's formulae still work.  It
# can be reset to different values with -x nmsvalue.
# Similarly, nbits "should" 24, but we can really make it anything
# we want that's less than or equal to rmax_bits = 32.  It can be
# reset to a new valu

##### Other options

Para ver otras opciones consultar el manual de la libreria[1].

Antes de pasar a realizar las pruebas sobre el generador es necesario explicar que debemos ser conscientes de que la evaluación proporcionada por dieharder en su informe estándar no debe darse por absolutamente cierta y tratarla con algo de sospecha. Es completamente posible que un generador pase todas las pruebas en lo que concierne a sus *p-values* individuales y que, sin embargo, fracase por completo al considerarlos todos juntos. De manera similar, es probable que al menos un *rng* aparezca como "débil" en 0, 1 o 2 pruebas en una ejecución con los valores por defecto de Dieharder: ```dieharder -a (ll)```, e incluso puede fallar 1 prueba en una de estas ejecuciones, en 10 o más. Para entender por qué esto es así, es necesario entender algo sobre las pruebas *rng*, los *p-values* y la hipótesis nula.

## Resultados de los tests de Diehard para MINSTD

A continuación probamos el generador MINSTD con los test de Diehard de la libreria Dieharder. Utilizamos tres semillas diferentes elegidas de forma aleatoria, y para cada una de ellas realizamos los test con 100 y 1000 *p-samples*.

In [4]:
SEED=19597 # SEED=$RANDOM en la ejecucion original. El rango de $RANDOM
# es menor que (1,2^31 -1)

In [9]:
echo $SEED

19597


##### Resultados en Dieharder con las opciones por defecto

In [10]:
dieharder -d 0 -g minstd -S $SEED -s 1

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -D 504 # Flags chosen so it
    # does not output the header again for each test
done    
    
    # dieharder can run single tests or all at once, nothing inbetween,
    # so we need this for loop to simulate the output that it would
    # produce for the diehahard tests if we run all tests in the library (flag -a)

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.26e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|     100|0.76137395|  PASSED  |     19597
      diehard_operm5|   0|   1000000|     100|0.80568945|  PASSED  
  diehard_rank_32x32|   0|     40000|     100|0.20973519|  PASSED  
    diehard_rank_6x8|   0|    100000|     100|0.78915130|  PASSED  
   diehard_bitstream|   0|   2097152|     100|0.65475181|  PASSED  
        diehard_opso|   0|   2097152|     100|0.00213706|   WEAK   
        diehard_oqso|   0|   2097152|     

##### Mismos tests pero con más p-samples

In [11]:
dieharder -d 0 -g minstd -S $SEED -s 1 -m 10

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -m 10 -D 504
done

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.41e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|    1000|0.66967787|  PASSED  |     19597
      diehard_operm5|   0|   1000000|    1000|0.91500707|  PASSED  
  diehard_rank_32x32|   0|     40000|    1000|0.27058043|  PASSED  
    diehard_rank_6x8|   0|    100000|    1000|0.54677804|  PASSED  
   diehard_bitstream|   0|   2097152|    1000|0.00417225|   WEAK   
        diehard_opso|   0|   2097152|    1000|0.00000000|  FAILED  
        diehard_oqso|   0|   2097152|    1

In [2]:
SEED=9779 # SEED=$RANDOM

echo $SEED

9779


##### Resultados en Dieharder con las opciones por defecto

In [2]:
##### Resultados en Dieharder con las opciones por defecto

dieharder -d 0 -g minstd -S $SEED -s 1

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -D 504
done

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.38e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|     100|0.53702723|  PASSED  |      9779
      diehard_operm5|   0|   1000000|     100|0.44726167|  PASSED  
  diehard_rank_32x32|   0|     40000|     100|0.45726413|  PASSED  
    diehard_rank_6x8|   0|    100000|     100|0.39219671|  PASSED  
   diehard_bitstream|   0|   2097152|     100|0.50762715|  PASSED  
        diehard_opso|   0|   2097152|     100|0.00497088|   WEAK   
        diehard_oqso|   0|   2097152|     

##### Mismos tests pero con más p-samples

In [3]:
##### Mismos tests pero con más p-samples

dieharder -d 0 -g minstd -S $SEED -s 1 -m 10

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -m 10 -D 504
done

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.41e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|    1000|0.20980324|  PASSED  |      9779
      diehard_operm5|   0|   1000000|    1000|0.50331258|  PASSED  
  diehard_rank_32x32|   0|     40000|    1000|0.76785019|  PASSED  
    diehard_rank_6x8|   0|    100000|    1000|0.24854251|  PASSED  
   diehard_bitstream|   0|   2097152|    1000|0.25445493|  PASSED  
        diehard_opso|   0|   2097152|    1000|0.00000000|  FAILED  
        diehard_oqso|   0|   2097152|    1

In [3]:
SEED=7721 # SEED=$RANDOM

echo $SEED

7721


##### Resultados en Dieharder con las opciones por defecto

In [2]:
##### Resultados en Dieharder con las opciones por defecto

dieharder -d 0 -g minstd -S $SEED -s 1

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -D 504
done    

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.27e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|     100|0.67609753|  PASSED  |      7721
      diehard_operm5|   0|   1000000|     100|0.54069778|  PASSED  
  diehard_rank_32x32|   0|     40000|     100|0.81753433|  PASSED  
    diehard_rank_6x8|   0|    100000|     100|0.27062161|  PASSED  
   diehard_bitstream|   0|   2097152|     100|0.42059845|  PASSED  
        diehard_opso|   0|   2097152|     100|0.00149632|   WEAK   
        diehard_oqso|   0|   2097152|     

##### Mismos tests pero con más p-samples

In [3]:
##### Mismos tests pero con más p-samples

dieharder -d 0 -g minstd -S $SEED -s 1 -m 10

for i in {1..16}; do
    dieharder -g minstd -d $i -S $SEED -s 1 -m 10 -D 504
done    

#=============================================================================#
#            dieharder version 3.31.1 Copyright 2003 Robert G. Brown          #
#=============================================================================#
   rng_name    |rands/second|
         minstd|  1.40e+08  |
#=============================================================================#
        test_name   |ntup| tsamples |psamples|  p-value |Assessment|    Seed  
#=============================================================================#
   diehard_birthdays|   0|       100|    1000|0.65795985|  PASSED  |      7721
      diehard_operm5|   0|   1000000|    1000|0.51244602|  PASSED  
  diehard_rank_32x32|   0|     40000|    1000|0.22232533|  PASSED  
    diehard_rank_6x8|   0|    100000|    1000|0.06927361|  PASSED  
   diehard_bitstream|   0|   2097152|    1000|0.06247225|  PASSED  
        diehard_opso|   0|   2097152|    1000|0.00000000|  FAILED  
        diehard_oqso|   0|   2097152|    1

En primer lugar, es necesario saber con que rangos de valores para *p-value* considera la librería Dieharder que las pruebas son "PASSED", "WEEK" o "FAILED". El criterio interno es actualmente: con valores de p < 0,5% o p > 99,5% los califíca como "WEAK"; con valores de p < 0,05% o p > 99,95%, "FAILED"; y cualquier otro valor de p, "PASSED".

En segundo lugar, también debemos explicar que dentro de la librería hay algunos test que no son considerados fiables para probar los generadores. ¿Por qué ocurre esto? Como se explica en la documentación de la librería, carecer de una fuente de números aleatorios perfectos para usar como referencia, hace que validar las pruebas no sea una tarea fácil y siempre deja cierta ambigüedad respecto a la verdadera valicez de estas pruebas. Durante el desarrollo de la librería, la validación de estos test se llevó a cabo utilizando varios generadores que son considerados por razones teóricas como "muy buenos generadores de números aleatorios". Al usar varios generadores de este tipo y no solo uno, se puede esperar que esos generadores tengan (al menos) correlaciones diferentes y que no todos deban fallar en una prueba de la misma manera y con el mismo número de p-value. Cuando todos estos generadores fallan constantemente en una prueba a un nivel dado, se sospecha que el problema está en el código de prueba y no en los generadores, aunque es muy difícil estar seguro.

Una de las ventajas de dieharder es que cuenta con una serie de estos "muy buenos generadores" disponibles. Para la validación de Dieharder se utilizó AES_OFB, Threefish_OFB, mt19937_1999, gfsr4, ranldx2 y taus2 (así como los números "random random" de random.org). Las pruebas (como la prueba de operm5 y sumas) que fallan constantemente con estos generadores están marcadas como "sospechosas" y sus resultados no deben usarse para probar generadores de números aleatorios.

Volviendo a los resultados de nuestras pruebas, podemos observar que con *p-samples* = 100 en los 3 casos ocurre lo mismo: pasan todos los test, ya que aquellos marcados como "WEAK" o "FAILED" en estos casos son en los test marcados como "sospechosos". Por otro lado, con *p-samples* = 1000 los resultados cambian.

Primero, ¿qué son los p-samples? Como ya hemos explicado en apartados anteriores, cada test obtiene un valor de *p-value*, en el caso de que la muestra del generedor sea aleatoria este valor debería seguir cierta distribución. En el caso de nuestros test, esa distribución es la uniforme. Con un único valor de *p-value* no es posible decir si la muestra es aleatoria realmente, ya que al ser una distribución uniforme puede ser cualquier valor en (0,1) con la misma probabilidad. Por eso, es necesario repetir el test varias veces y obtener varios *p-values* y comprobar que en efecto siguen una distribución uniforme. Para hacer esas repeticiones de forma automática, la librería Dieharder utiliza el parametro *p-samples*. Por defecto el valor de *p-samples*(repeticiones de *p-values*) es de 100. A partir de todos los *p-values* obtenidos en cada *p-sample* se obtiene un último y único *p-value* que se evalua mediante el test de Kolmogorov-Smirnov para determinar si el test pasa o falla. Cuanto mayor sea el número de *p-samples*, con mejor precisión se puede estimar si realmente siguen la distribución deseada, pero mayor es el coste computacional.

En nuestro caso, podemos ver que al aumentar los test *parking_lot*, *2dsphere* y *3dsphere* se vuelven débiles o fallan. Sabemos que hoy en día se ha demostrado que existen mejores generadores que MINSTD, que como vemos en nuestras pruebas no es capaz de pasar todos los test conforme aumenta el número de *p-samples*. Sin embargo, se sigue utilizando en algunos estándares porque sí que es capaz de generar secuencias de números aleatorios para muestras que no sean de un tamaño considerable. Es decir, que con el es posible obtener suficientes números aleatorios para experimentos que no necesiten cantidades muy grandes de estos números.

## Descripción del algunos tests

En los puntos anteriores se han lanzado los test alternando valores de 3 *Seeds* diferentes y *psamples* de dimensiones 100 y 1000. A partir de los resultados obtenidos, vamos a centrarnos en varios de estos casos y explicarlos. Hemos elegido los siguientes:

- **1. Diehard Overlapping Permutations Test**
-  **2. Diehard Minimum Distance Test: 3d Sphere**
-  **3. Diehard Runs Test**

La diferencia fundamental de estos test radica en la función que permite obtener el p-valor, dato fundamental ya que determina si el test falla cuando el valor obtenido es muy poco probable bajo esa distribución.


#### 1 - Diehard Overlapping Permutations Test

Este test trabaja con una secuencia de un millón de enteros aleatorios a partir de los cuales determina el orden de
grupos de 5 enteros, en orden ascendente o descendente.  A partir de este punto cuenta el número de las 120 posibles 
configuraciones (5!) y define la varianza para el resultado esperado de estos valores.

In [10]:
dieharder -d 1 -h


#==================================================================
#          Diehard Overlapping 5-Permutations Test.
# This is the OPERM5 test.  It looks at a sequence of one mill- 
# ion 32-bit random integers.  Each set of five consecutive     
# integers can be in one of 120 states, for the 5! possible or- 
# derings of five numbers.  Thus the 5th, 6th, 7th,...numbers   
# each provide a state. As many thousands of state transitions  
# are observed,  cumulative counts are made of the number of    
# occurences of each state.  Then the quadratic form in the     
# weak inverse of the 120x120 covariance matrix yields a test   
# equivalent to the likelihood ratio test that the 120 cell     
# counts came from the specified (asymptotically) normal dis-   
# tribution with the specified 120x120 covariance matrix (with  
# rank 99).  This version uses 1,000,000 integers, twice.       
#
# Note that Dieharder runs the test 100 times, not twice, by
# default.
#========================

Los resultados comparativos para este test en los casos que hemos presentado anteriormente son los siguientes:

In [ ]:
#=============================================================================#
       test_name   |ntup| tsamples |psamples|  p-value |Assessment
#=============================================================================#

## Seed 19597
### diehard_operm5|   0|   1000000|     100|0.80568945|  PASSED  
### diehard_operm5|   0|   1000000|    1000|0.91500707|  PASSED  

## Seed 9779
### diehard_operm5|   0|   1000000|     100|0.44726167|  PASSED  
### diehard_operm5|   0|   1000000|    1000|0.50331258|  PASSED

## Seed 7721
### diehard_operm5|   0|   1000000|     100|0.54069778|  PASSED
### diehard_operm5|   0|   1000000|    1000|0.51244602|  PASSED


#### 2 - Diehard Minimum Distance Test: 3d Sphere

Este test elige 4000 puntos aleatorios que se localizan en un cubo de 1000x1000. En cada punto se localiza una esfera suficientemente grande de manera que toque al punto más cercano. Se busca el radio más pequeño y se repite el test 20 veces. Estos radios deberían estar distribuídos exponencialmente y el pvalor será determinado en función de como de lejos están los resultados de los esperados.

In [22]:
dieharder -d 12 -h

#==================================================================
#          Diehard 3d Sphere (Minimum Distance) Test
# Choose  4000 random points in a cube of edge 1000.  At each   
# point, center a sphere large enough to reach the next closest 
# point. Then the volume of the smallest such sphere is (very   
# close to) exponentially distributed with mean 120pi/3.  Thus  
# the radius cubed is exponential with mean 30. (The mean is    
# obtained by extensive simulation).  The 3DSPHERES test gener- 
# ates 4000 such spheres 20 times.  Each min radius cubed leads 
# to a uniform variable by means of 1-exp(-r^3/30.), then a     
#  KSTEST is done on the 20 p-values.                           
#
# This test ignores tsamples, and runs the usual default 100
# psamples to use in the final KS test.
#==================================================================


Los resultados comparativos para este test en los casos que hemos presentado anteriormente son los siguientes:

In [ ]:
#=============================================================================#
       test_name   |ntup| tsamples |psamples|  p-value |Assessment
#=============================================================================#

## Seed 19597
### diehard_3dsphere|   3|      4000|     100|0.82592641|  PASSED
### diehard_3dsphere|   3|      4000|    1000|0.00045960|   WEAK

## Seed 9779
### diehard_3dsphere|   3|      4000|     100|0.57506385|  PASSED
### diehard_3dsphere|   3|      4000|    1000|0.00603629|  PASSED

## Seed 7721
### diehard_3dsphere|   3|      4000|     100|0.22667993|  PASSED
### diehard_3dsphere|   3|      4000|    1000|0.00000697|   WEAK

#### 3 - Diehard Runs Test

En este test se itera sobre 10.000 valores y se definen consecutivas ejecuciones en las que los elementos van aumentando o decreciendo su valor de manera consecutiva y se ejecuta 10 veces. Para obtener los resultados ésto se compara con una matriz de covarianza.

In [28]:
dieharder -d 15 -h

#==================================================================
#                    Diehard Runs Test
#  This is the RUNS test.  It counts runs up, and runs down, 
# in a sequence of uniform [0,1) variables, obtained by float-  
# ing the 32-bit integers in the specified file. This example   
# shows how runs are counted:  .123,.357,.789,.425,.224,.416,.95
# contains an up-run of length 3, a down-run of length 2 and an 
# up-run of (at least) 2, depending on the next values.  The    
# covariance matrices for the runs-up and runs-down are well    
# known, leading to chisquare tests for quadratic forms in the  
# weak inverses of the covariance matrices.  Runs are counted   
# for sequences of length 10,000.  This is done ten times. Then 
# repeated.                                                     
#
# In Dieharder sequences of length tsamples = 100000 are used by
# default, and 100 p-values thus generated are used in a final
# KS test.
#=======================================

Los resultados comparativos para este test en los casos que hemos presentado anteriormente son los siguientes:

In [ ]:
#=============================================================================#
       test_name   |ntup| tsamples |psamples|  p-value |Assessment
#=============================================================================#

## Seed 19597
### diehard_runs|   0|    100000|     100|0.96717776|  PASSED
### diehard_runs|   0|    100000|    1000|0.99083300|  PASSED
## Seed 9779
### diehard_runs|   0|    100000|     100|0.50758671|  PASSED
### diehard_runs|   0|    100000|    1000|0.08828048|  PASSED
## Seed 7721
### diehard_runs|   0|    100000|     100|0.61301442|  PASSED
### diehard_runs|   0|    100000|    1000|0.95153922|  PASSED

### Materiales y referencias

[1] http://webhome.phy.duke.edu/~rgb/General/dieharder.php

[2] https://www.systutorials.com/docs/linux/man/1-dieharder/

[3] Random number generators: good ones are hard to find - Park and Miller, 1988

[4] George Marsaglia, Wai Wan Tsang, Some Difficult-to-pass Tests of Randomness, Journal of Statistical Software, Volume 7, 2002, Issue 3.

[5] PDF tema 2 de clase